# RAG
Retrieval Augmented Generation, 검색 증강 생성

LLM에게 외부 데이터를 context로 활용할 수 있도록 처리 하는 과정

## 사전처리 단계

1. 문서 불러오기(Loading)
2. 텍스트 나누기(Splitting)
3. 숫자로 바꾸기(Embedding)
4. 저장하기(VectorStore)

In [3]:
from dotenv import load_dotenv

load_dotenv()

True

In [ ]:
# 예시
from langchain_core.documents import Document

documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Goldfish are popular pets for beginners, requiring relatively simple care.",
        metadata={"source": "fish-pets-doc"},
    ),
    Document(
        page_content="Parrots are intelligent birds capable of mimicking human speech.",
        metadata={"source": "bird-pets-doc"},
    ),
    Document(
        page_content="Rabbits are social animals that need plenty of space to hop around.",
        metadata={"source": "mammal-pets-doc"},
    ),
]  # 문서 불러오기 - 텍스트 나누기(1, 2)까지 완료된 상태

In [3]:
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

# OpenAI의 임베딩 모델
embedding = OpenAIEmbeddings(model='text-embedding-3-small')
embedding.embed_query('고양이보다는 개가 더 좋지')

# Vector Store에 임베딩 후 저장(3, 4) - 메모리에
vectorstore = Chroma.from_documents(documents, embedding=embedding)

In [6]:
vectorstore.similarity_search(
    '고양이보다 개가 더 좋아', 
    k=2
)

[Document(id='2048dff6-c685-4321-aee2-599600359c6a', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
 Document(id='387917e9-8ec1-4d18-b27e-d904ee4239b5', metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are popular pets for beginners, requiring relatively simple care.')]

In [18]:
# 1. Load
from langchain_community.document_loaders import PyMuPDFLoader

loader = PyMuPDFLoader('../data/spri.pdf')
docs = loader.load()

print(len(docs))

# for doc in docs:
#     print(doc)

23


In [19]:
# 2. Split
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Recursive~: 대충 자르는 splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,  # 글자 수
    chunk_overlap=50  # 두 청크 사이에 겹치는 글자 수를 50개
)

docs_splitted = text_splitter.split_documents(docs)
print(len(docs_splitted))

# for doc in docs_splitted:
#     print(doc)
#     print('========================================================')

72


In [ ]:
# 3. 벡터 임베딩 + vectorstore 저장
from langchain_openai import OpenAIEmbeddings  # 3. 임베딩 담당
from langchain_community.vectorstores import FAISS  # 4. 벡터스토어 담당

embedding = OpenAIEmbeddings()

vectorstore = FAISS.from_documents(documents=docs_splitted, embedding=embedding)

vectorstore.similarity_search(
    '미국 대통령과 관련된 문서들 가져와봐'
)

[Document(id='e1df93b2-8c5a-4acb-9f87-62822783df02', metadata={'producer': 'Hancom PDF 1.3.0.542', 'creator': 'Hwp 2018 10.0.0.13462', 'creationdate': '2023-12-08T13:28:38+09:00', 'source': '../data/spri.pdf', 'file_path': '../data/spri.pdf', 'total_pages': 23, 'format': 'PDF 1.4', 'title': '', 'author': 'dj', 'subject': '', 'keywords': '', 'moddate': '2023-12-08T13:28:38+09:00', 'trapped': '', 'modDate': "D:20231208132838+09'00'", 'creationDate': "D:20231208132838+09'00'", 'page': 3}, page_content='1. 정책/법제  \n2. 기업/산업 \n3. 기술/연구 \n 4. 인력/교육\n미국, 안전하고 신뢰할 수 있는 AI 개발과 사용에 관한 행정명령 발표 \nn 미국 바이든 대통령이 ‘안전하고 신뢰할 수 있는 AI 개발과 사용에 관한 행정명령’에 서명하고 \n광범위한 행정 조치를 명시\nn 행정명령은 △AI의 안전과 보안 기준 마련 △개인정보보호 △형평성과 시민권 향상 △소비자 \n보호 △노동자 지원 △혁신과 경쟁 촉진 △국제협력을 골자로 함\nKEY Contents\n£ 바이든 대통령, AI 행정명령 통해 안전하고 신뢰할 수 있는 AI 개발과 활용 추진\nn 미국 바이든 대통령이 2023년 10월 30일 연방정부 차원에서 안전하고 신뢰할 수 있는 AI 개발과 \n사용을 보장하기 위한 행정명령을 발표\n∙행정명령은 △AI의 안전과 보안 기준 마련 △개인정보보호 △형평성과 시민권 향상 △소비자 보호 \n△노동자 지원 △혁신과 경쟁 촉진 △국제협력에 관한 내용을 포괄'),
 Do

## 검색 증강 단계

1. 사용자 질문(Query)
2. 검색(Retrieve)
3. LLM
4. 최종 답변

In [ ]:
from langchain import hub
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# 프롬프트(langchain hub에서)
prompt = hub.pull('rlm/rag-prompt')

# LLM(gpt)
llm = ChatOpenAI(model='gpt-4.1-nano')

# 검색기(Retriever)
retriever = vectorstore.as_retriever()

chain = (
    {'context': retriever, 'question': RunnablePassthrough()}  # 들어온 인자가 그대로 들어옴
    | prompt
    | llm
    | StrOutputParser()
)

chain.invoke('삼성전자 관련 소식을 다 가져와줘')

'삼성전자는 자체 개발한 생성 AI인 ‘삼성 가우스’를 공개했으며, 언어, 코드, 이미지 모델로 구성되어 온디바이스에서 작동합니다. 이 AI는 외부 유출 위험이 낮고, 스마트폰 등 다양한 제품에 단계적으로 탑재될 예정입니다. 2023년 11월 ‘삼성 AI 포럼 2023’에서 최초 공개되었으며, 향후 경쟁 제품들과의 차별화를 기대할 수 있습니다.'

In [5]:
# 관심있는 데이터 소스로 RAG 해보기
from langchain_community.document_loaders import PyMuPDFLoader  # 1. 데이터 로드 담당
from langchain_text_splitters import RecursiveCharacterTextSplitter  # 2. 데이터 스플릿 담당
from langchain_openai import OpenAIEmbeddings  # 3. 임베딩 담당
from langchain_community.vectorstores import Chroma  # 4. 벡터스토어 담당

# 1. 데이터 로딩
loader = PyMuPDFLoader('../data/Organic_Chemistry.pdf')
docs = loader.load()

print(len(docs))

# 2. 데이터 스플릿
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,  # 글자 수
    chunk_overlap=50  # 두 청크 사이에 겹치는 글자 수를 50개
)

docs_splitted = text_splitter.split_documents(docs)
print(len(docs_splitted))

# 3. 임베딩
embedding = OpenAIEmbeddings()

# 4. 벡터스토어
vectorstore = Chroma.from_documents(documents=docs_splitted, embedding=embedding)

1347
6519


In [9]:
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from langchain_tavily import TavilySearch

# 책 분석 프롬프트
book_prompt = PromptTemplate(
    input_variables=['query', 'book'],
    template='''
당신은 질문에 응답하는 작업을 하기 위한 유능한 도우미입니다.
주어진 책에서 검색된 내용을 바탕으로, 사용자의 질문에 대한 답변을 작성하세요.
최대 5문장까지 답변하고, 답변을 간결하게 유지하세요.
답을 모르겠다면, 그냥 답을 모르겠다고 말하세요.

질문: {query}

책 내용: {book}

답변:
''')

# 웹 검색 프롬프트
web_prompt = PromptTemplate(
    input_variables=['book_analysis'],
    template='''
당신은 질문에 응답하는 작업을 하기 위한 유능한 도우미입니다.
아래 책 분석 내용을 바탕으로 추가 정보를 얻기 위한 검색어를 생성하세요.
검색어는 짧고 간결하게 하나만 만드세요.
답을 모르겠다면, 그냥 답을 모르겠다고 말하세요.

책 분석 내용: {book_analysis}

답변:
''')

# 최종 프롬프트
final_prompt = PromptTemplate(
    input_variables=['query', 'book_analysis', 'web_search'],
    template='''
당신은 질문에 응답하는 작업을 하기 위한 유능한 도우미입니다.
다음은 책 분석과 웹 검색 결과입니다. 이 두 정보를 바탕으로, 사용자의 질문에 대한 최종 답변을 작성하세요.
답을 모르겠다면, 그냥 답을 모르겠다고 말하세요.

질문: {query}

책 분석 내용: {book_analysis}

웹 분석 내용: {web_search}

답변:
''')

# LLM(gpt)
llm = ChatOpenAI(model='gpt-4.1-nano', temperature=0)

# 검색기(Retriever)
retriever = vectorstore.as_retriever()

# 웹검색
search_tool = TavilySearch()

# 웹 검색 결과 포맷
format_web_results = RunnableLambda(
    lambda x: "\n".join([f"- {r['title']}: {r['content']}" for r in x["results"]])
)


# book chain
book_chain = (
    {
        "query": RunnablePassthrough(),
        "book": retriever
    }
    | book_prompt
    | llm
    | StrOutputParser()
)

# web chain
web_chain = (
    book_chain
    | web_prompt
    | llm
    | StrOutputParser()
    | search_tool
    | format_web_results
)

# 전체 체인
chain = (
    {
        "query": RunnablePassthrough(),
        "book_analysis": book_chain,
        "web_search": web_chain
    }
    | final_prompt
    | llm
    | StrOutputParser()
)

chain.invoke('이 책의 저자의 생애가 궁금해')

'이 책의 저자인 Francis A. Carey와 Richard J. Sundberg의 생애에 대한 구체적인 정보는 제공된 자료에 포함되어 있지 않습니다. 따라서 저자들의 생애에 대해 알 수 있는 내용이 없습니다. 추가적인 자료가 필요할 것 같습니다.'

## Agent + RAG

In [17]:
from langchain.agents import create_openai_tools_agent, AgentExecutor
from langchain.chat_models import init_chat_model
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferMemory
from datetime import datetime

from langchain_community.document_loaders import PyMuPDFLoader  # 1. 데이터 로드 담당
from langchain_text_splitters import RecursiveCharacterTextSplitter  # 2. 데이터 스플릿 담당
from langchain_openai import OpenAIEmbeddings  # 3. 임베딩 담당
from langchain_community.vectorstores import FAISS  # 4. 벡터스토어 담당
from langchain.tools.retriever import create_retriever_tool

today = datetime.today().strftime('%D')

llm = ChatOpenAI(model='gpt-4.1-nano')

loader = PyMuPDFLoader('../data/spri.pdf')
docs = loader.load()

splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
split_docs = splitter.split_documents(docs)

embedding = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(split_docs, embedding=embedding)
retriever = vectorstore.as_retriever()

rag_tool = create_retriever_tool(
    retriever,
    name='pdf_search',
    description='PDF 문서에서 질문과 관련된 내용을 검색합니다'  # agent가 언제 이 툴을 쓸지 알게 된다
)

text = f'''
너는 웹 검색도 가능하고, 2023년 12월 인공지능 산업 최신동향 정보를 담은 pdf도 검색할 수 있는 어시스턴트야.

사용자가 pdf 문서와 관련된 질문(ex. 'pdf에서', '문서 내용', '파일에서')을 하면 반드시 pdf_search 도구를 써야 해

오늘 날짜는 미국식 날짜 표기로 {today}야. 최신 정보에 관한 정보가 필요하다면, 이 날짜를 기준으로 생각해.

사용자가 최신성이나 팩트 체크를 필요하는 질문을 한 경우, web_search를 실행해야 해.

확실하게 판단이 되지 않는다면, web_search와 pdf_search 둘 다 사용해.

사용자가 일반적인 질문을 하거나, 최신성 혹은 팩트 체크가 필요하자 않다면 도구를 사용하지 않아도 좋아.

만약 답을 모르겠다면, 그냥 답을 모르겠다고 말해.

가장 의미있는 검색결과들을 요약해서 질문에 답해줘.

'''

prompt = ChatPromptTemplate.from_messages([
    ('system', text),
    MessagesPlaceholder(variable_name='chat_history'),
    ('human', '{input}'),
    MessagesPlaceholder(variable_name='agent_scratchpad')
])

memory = ConversationBufferMemory(
    return_messages=True,
    memory_key='chat_history'
)

agent = create_openai_tools_agent(
    llm=llm,
    tools=[search_tool, rag_tool],
    prompt=prompt,
)

agent_executer = AgentExecutor(agent=agent, memory=memory, tools=[search_tool, rag_tool], verbose=True)

In [19]:
agent_executer.invoke({'input': '지금 정보는 2023년 기준인데, 관련된 최신 동향을 보고 싶어'})



> Entering new AgentExecutor chain...

Invoking: `tavily_search` with `{'query': '삼성전자 인공지능 산업 최신 동향 2023년 이후', 'search_depth': 'advanced', 'time_range': 'month'}`


{'query': '삼성전자 인공지능 산업 최신 동향 2023년 이후', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'url': 'https://www.eugenefn.com/common/files/amail//20250806_B_isbang_223.pdf', 'title': 'AI Quant', 'content': 'SK하이닉스 56_ Eugene Research Center 2024 년 하반기에 두 기업의 심리는 큰 변화를 맞이하였다. 먼저8 월 고점을 기 점으로 삼성전자에 대한 애널리스트 심리가 꺾이기 시작하여, 하반기 동안 가파 르게 하락세를 보였다. 특히 2024 년 10~11 월경 애널리스트 보고서에서 부정 적 톤이 쏟아지며 심리 점수가 급락하였고, 2024 년 말에는 중립(0)에 가까운 수 준까지 하락하였다. 이는 아마도 하반기 실적 악화나 업황 둔화에 대한 우려가 본격 반영된 결과로 해석된다. 이후 2025 년에 들어서 삼성전자 주가가 회복 국면을 보였지만, 애널리스트 심 리는 이전만큼 회복되지 않고 중립 부근의 신중한 수준에 머물렀다. 2025 년 3~4 월 일부 호전되어 +0.07 까지 상승, 2 분기 실적 컨퍼런스 이후 +0.15 로 큰 폭 상승하며 회복세를 지속하고 있다. SK하이닉스 애널리스트 심리 추이는 더욱 극적인 저점 대비 반등 양상을 보였다. 앞서 언급했듯 2023 [...] 지표이다. [Dr. AI Quant] 애널리스트 심리 대전: 삼성전자VS. SK하이닉스 42_ Eugene Research Center 2023 년 이후 구축된 삼성전자의 애널리

{'input': '지금 정보는 2023년 기준인데, 관련된 최신 동향을 보고 싶어',
 'chat_history': [HumanMessage(content='pdf에서 삼성전자 관련 내용을 요약해줘.', additional_kwargs={}, response_metadata={}),
  AIMessage(content='삼성전자에 관한 내용은 다음과 같습니다:\n\n1. **생성형 인공지능 ‘삼성 가우스’ 공개**: 삼성전자는 2023년 11월 8일 ‘삼성 AI 포럼 2023’에서 자체 개발한 생성형 AI인 ‘삼성 가우스’를 공개하였으며, 이 모델은 온디바이스에서 작동 가능하고, 언어, 코드, 이미지를 각각 담당하는 3개 모델로 구성되어 있습니다. 이는 외부로 사용자 정보 유출 위험이 없으며, 단계적으로 다양한 제품에 탑재될 계획입니다.\n\n2. **삼성 가우스의 특징**:\n   - 언어, 코드, 이미지 세 가지 모델로 구성\n   - 클라우드와 온디바이스용 모델 제공, 메일 작성, 문서 요약, 번역 지원\n   - AI 코딩 어시스턴트 ‘코드아이’를 통해 소프트웨어 개발 최적화\n   - 이미지를 창의적으로 생성하고 편집하며, 저해상도 이미지를 고해상도로 전환하는 기능 포함\n   - 라이선스 또는 개인정보 침해 없이 학습된 안전한 데이터 활용\n\n3. **제품과 기술 적용 계획**: 삼성전자는 ‘삼성 가우스’를 다양한 제품에 단계적으로 탑재하며, 향후 온디바이스 AI 기술 강화에 주력할 예정입니다.\n\n4. **기타 관련 행사**:\n   - 2024년 1월 9일~12일, 미국 라스베이거스에서 CES 2024가 열리며, AI를 포함한 최신 기술들이 전시될 예정입니다.\n   - AI와 머신러닝 관련 국제 컨퍼런스인 AIMLA 2024도 개최 예정입니다.\n\n이 내용은 삼성전자의 최신 AI 개발 및 관련 기술 적용 계획에 대한 정보를 포함하고 있습니다.', additional_kwargs={}, response_metadata={}),
  Huma